In [1]:
#generate for bar chart
from gensim.models.keyedvectors import KeyedVectors
#from gensim.models import word2vec
import json
import numpy as np
from konlpy.tag import Kkma
from konlpy.utils import pprint
import math
import csv
import codecs
l_quality= open("LectureQuality.txt", "r", encoding="utf-8").readlines()
load= open("Load.txt", "r", encoding="utf-8").readlines()
grade= open("Grade.txt", "r", encoding="utf-8").readlines()
int_friendly= open("InternationalFriendly.txt", "r", encoding="utf-8").readlines()
CATEGORY_COUNT = 3

c:\users\minkyu\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
pos_vectors = KeyedVectors.load_word2vec_format('../pos_naver_news.vec', binary=False)

In [3]:
scores = []
subjects = [l_quality, load, grade]
def add_to_dict(subject):
    subject_scores={}
    for i in range(len(subject)):
        p = subject[i].split("\t")
#         print(p)
        subject_scores[p[0]] = float(p[1][:-1])
    scores.append(subject_scores)

for subject in subjects:
    add_to_dict(subject)

In [4]:
subject_mp = {"강의":0,"널널":1,"성적":2}
def calculate_score(s, subject_str):
    result = 0
    for k,v in scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

In [5]:
REVIEWS = "cs_reviews"

f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

scores_list = []
subjects_str = ["강의", "널널", "성적"]
for subject_str in subjects_str:
    scores_list_subject=[]
    for i in range(len(f[:])):
        line = f[i]
        course = json.loads(line)
        scores_list_subject.append(calculate_score(course["review_text"], subject_str))
    scores_list.append(scores_list_subject)


In [6]:
f_count_word = {}
count_word = {}
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}
#num_word={}

def calc_weight(s):
    return (mp[s]) - 3 

for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["성적"] == "?":
        continue
    words = course["review_text"].split()
    for w in words:
        if w not in count_word:
            count_word[w] = 0
            #num_word[w]=0
        #num_word[w]+=1
        count_word[w] += calc_weight(course["성적"])
# for w in count_word:
#     count_word[w]=float(count_word[w])/float(num_word[w])


In [8]:
sorted_by_value = sorted(count_word.items(), key=lambda kv: kv[1])[::-1]
# print(sorted_by_value)

In [9]:
# sorted_by_value

In [10]:
def calc_score_avg(column_name,comment_id):
    i = comment_id
    course = json.loads(f[i])
    #print(mp[course["강의"]],(scores_list[i]))
    return scores_list[subject_mp[column_name]][i]
    

for i in range(len(f)):
    course = json.loads(f[i])
    print(course["course_code"])
    
    for x in subjects_str:
        print(x,calc_score_avg(x,i))
    break
    

CS322
강의 0.8336546070121553
널널 -0.6640367702678489
성적 0.992631520201128


In [13]:
def append_noun(s):
    return "('%s','Noun')" % (s)

naver_wv_categories = ["강의", "부담", "성적"]
wv_update = {}
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        if append_noun(k) in pos_vectors.vocab:
#             print(k,naver_wv_categories[i])
#             print(pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i])))
            wv_update[k] = pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i]))
        

c:\users\minkyu\appdata\local\programs\python\python35\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [27]:
review_update = {}
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)

    for i in range(CATEGORY_COUNT):
        if course[subjects_str[i]] == "?":
            continue
        for k,v in scores[i].items():
            if k in course["review_text"].split():
                if k not in review_update:
                    review_update[k] = 0
                review_update[k] += calc_weight(course[subjects_str[i]])*100.0/len(course["review_text"])
        

In [28]:
aggregated_scores = [{} for i in range(CATEGORY_COUNT)]
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        aggregated_scores[i][k] = v
        if k in review_update:
            aggregated_scores[i][k] += review_update[k]*0.1
        
        if k in wv_update:
            aggregated_scores[i][k] += wv_update[k]
            
            

In [29]:
def calc_aggregated_score(s, subject_str):
    result = 0
    for k,v in aggregated_scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

def calc_aggregated_score_sum(column_name,i):
    course = json.loads(f[i])
    return calc_aggregated_score(course["review_text"], column_name)

# print(subjects_str)

In [30]:
for i in range(len(f[:1])):
    course = json.loads(f[i])
    print(course)

{'종합': '?', '성적': '?', '널널': '?', 'review_text': '오랜만에 한국어 강의가 개설되어 재미있게 들었습니다.학점은 한국어 강의니까 빡시게 주신다고 했는데 예상했던 만큼 딱 나왔습니다. 다른 전공과 크게 다르지 않은 수준.거의 매주 손으로 풀어야하는 숙제가 나오고 프로그래밍 프로젝트도 4~5개 있었는데 미루지 않고 꼬박꼬박 하면 할만합니다. 코딩 플젝은 스펙이 매우 자유로워서 당황스러울 수 있는데 그만큼 채점도 너그럽게 하시는 것 같습니다. 시험 공부할 양도 아주 많진 않아요.교수님께서 강의를 잘하셔서(특히 한국어강의여서) 집중해서 듣기만 하면 이해도 잘되고 남는것도 많았습니다. 수업중 딴얘기도 종종 하시는데 저에게는 개인적으로 유익했지만 이건 호불호가 갈리는듯...', 'course_code': 'CS322', '추천': '0', 'url_id': 2888, 'year': '2012', 'semester': '가을', '강의': '?', 'prof_name': '최광무', 'course_name': '형식언어및오토마타'}


In [33]:
pos = {}
neg = {}
PNS = {}
course_list = []
print(subjects_str)

for i in range(len(f[:])):
    course = json.loads(f[i])
#     print(course["course_code"]+"\t",end="")
    if course["course_code"] not in pos:
        pos[course["course_code"]] = [[0,0],[0,0],[0,0]]
        neg[course["course_code"]] = [[0,0],[0,0],[0,0]]
        PNS[course["course_code"]] = [[0,0],[0,0],[0,0]]#[0,0,0]
        course_list.append(course["course_code"])
    for x in subjects_str:
#             out.write(str(calc_aggregated_score_sum(x,i))+"\t")
#         print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
        sc = calc_aggregated_score_sum(x,i)
        if sc>0:
            pos[course["course_code"]][subject_mp[x]][0] += sc*10
            pos[course["course_code"]][subject_mp[x]][1] += 1
        else:
            neg[course["course_code"]][subject_mp[x]][0] -= sc*10
            neg[course["course_code"]][subject_mp[x]][1] += 1
    for x in subjects_str:
#         pos_vir = pos[course["course_code"]][subject_mp[x]][1]
#         neg_vir = neg[course["course_code"]][subject_mp[x]][1]
#         if pos_vir == 0 and neg_vir == 0:
#             pos_vir = 1
        pos_sum = pos[course["course_code"]][subject_mp[x]][0]
#         pos_cnt = pos[course["course_code"]][subject_mp[x]][1]
        neg_sum = neg[course["course_code"]][subject_mp[x]][0]
#         neg_cnt = neg[course["course_code"]][subject_mp[x]][1]
        PNS[course["course_code"]][subject_mp[x]][0] = pos_sum-neg_sum

['강의', '널널', '성적']


In [34]:
print(neg["CS322"])

[[168.89481821880213, 22], [196.0387243496549, 23], [209.53253525372617, 23]]


In [35]:
with open("mingyu.txt","w",encoding="utf-8") as out:
    for k,v in pos.items():
        print(k)
        print(*pos[k])
        print(*neg[k])
        print(*PNS[k])
    
    

CS454
[0, 0] [0, 0] [0, 0]
[30.0, 3] [30.0, 3] [30.0, 3]
[-30.0, 0] [-30.0, 0] [-30.0, 0]
CS522
[0, 0] [0, 0] [0, 0]
[0.9274994044022122, 1] [9.999999990447275, 1] [9.999999968283838, 1]
[-0.9274994044022122, 0] [-9.999999990447275, 0] [-9.999999968283838, 0]
CS548
[0, 0] [0, 0] [0, 0]
[20.0, 2] [20.0, 2] [20.0, 2]
[-20.0, 0] [-20.0, 0] [-20.0, 0]
CS322
[5.329438803942934, 2] [1.5108351434541412, 1] [8.793556667391947, 1]
[168.89481821880213, 22] [196.0387243496549, 23] [209.53253525372617, 23]
[-163.5653794148592, 0] [-194.52788920620077, 0] [-200.7389785863342, 0]
CS457
[0, 0] [0, 0] [0, 0]
[10.0, 1] [10.0, 1] [10.0, 1]
[-10.0, 0] [-10.0, 0] [-10.0, 0]
CS453
[5.842192731929995, 1] [0, 0] [5.370495669980353, 1]
[29.99999999805195, 4] [46.18963983191598, 5] [35.37049566998036, 4]
[-24.157807266121957, 0] [-46.18963983191598, 0] [-30.000000000000004, 0]
CS550
[0, 0] [0, 0] [0, 0]
[9.936457881722868, 1] [9.999999999998117, 1] [9.999999968283838, 1]
[-9.936457881722868, 0] [-9.99999999999

In [37]:
for k,v in pos.items():
    with open("../data/"+k+".csv","w",newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerow(["Course","Pos","Neg"])
        for subject in subjects_str :
            if subject == "강의":
                line = ["Lecture",str(pos[k][0][0]),str(neg[k][0][0])]
            elif subject == "널널":
                line = ["Load",str(pos[k][1][0]),str(neg[k][1][0])]
            else:
                line = ["Grade",str(pos[k][2][0]),str(neg[k][2][0])]
            writer.writerow(line)

In [38]:
for i in range(len(f[:])):
    course = json.loads(f[i])
    str1 = course["course_code"] + course["year"] + course["semester"]
    print(str1)
    break
    

CS3222012가을


In [39]:
pos1 = {}
neg1 = {}
PNS1 = {}

for i in range(len(f[:])):
    course = json.loads(f[i])
    key1 = course["course_code"] +" "+ course["year"] +" "+ course["semester"]
    if course["semester"]!="봄" and course["semester"]!="가을":
        pass
    else:
        print(course["course_code"]+"\t",end="")
        if key1 not in pos1:
            pos1[key1] = [[0,0],[0,0],[0,0]]
            neg1[key1] = [[0,0],[0,0],[0,0]]
            PNS1[key1] = [[0,0],[0,0],[0,0]]#[0,0,0]
        for x in subjects_str:
    #             out.write(str(calc_aggregated_score_sum(x,i))+"\t")
            print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
            sc = calc_aggregated_score_sum(x,i)
            if sc>0:
                pos1[key1][subject_mp[x]][0] += sc*10
                pos1[key1][subject_mp[x]][1] += 1
            else:
                neg1[key1][subject_mp[x]][0] -= sc*10
                neg1[key1][subject_mp[x]][1] += 1
        print("")
        for x in subjects_str:
            pos_vir1 = pos1[key1][subject_mp[x]][1]
            neg_vir1 = neg1[key1][subject_mp[x]][1]
            if pos_vir1 == 0:
                pos_vir1 = 1
            if neg_vir1 == 0:
                neg_vir1 = 1
            pos_sum1 = pos1[key1][subject_mp[x]][0]
            pos_cnt1 = pos1[key1][subject_mp[x]][1]
            neg_sum1 = neg1[key1][subject_mp[x]][0]
            neg_cnt1 = neg1[key1][subject_mp[x]][1]
            PNS1[key1][subject_mp[x]][0] = pos_sum1-neg_sum1
            
            
            

CS322	-1.0	-1.0	-1.0	
CS493	-1.0	-1.0	-1.0	
CS322	-1.0	-1.0	-1.0	
CS311	-1.0	-1.0	-1.0	
CS230	-1.0	-1.0	-1.0	
CS300	-0.999999884066068	-1.0	-0.9999999999997665	
CS109	-0.9999918163395858	-0.9999999999999971	-1.0	
CS206	-0.37668098171047204	-0.9999999968283837	-1.0	
CS204	-1.0	-1.0	-1.0	
CS220	-1.0	-1.0	-1.0	
CS330	-1.0	-1.0	-1.0	
CS448	-0.9995789015521455	-0.9999999990447276	-0.9999999968283837	
CS211	-1.0	-1.0	-1.0	
CS230	-1.0	-1.0	-1.0	
CS202	0.9705418327323984	-0.6992147364151174	0.22985275596975746	
CS202	0.990790821492823	-0.9999999998234165	-1.0	
CS109	-0.9999903134635382	-0.9999999999999922	-1.0	
CS330	-0.37284367395320756	-0.6653566588558755	-0.9999999926656689	
CS311	-1.0	-1.0	-1.0	
CS320	-1.0	-1.0	-1.0	
CS204	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS230	0.22083527750341492	-0.9969461295136471	-0.8813689328912009	
CS360	-1.0	-1.0	-1.0	
CS300	-1.0	-1.0	-1.0	
CS330	-1.0	-1.0	-1.0	
CS330	-1.0	-1.0	-1.0	
CS322	-0.9936457881722868	-0.9999999999999419	-0.9999999990447276	
CS202	-1.0	

CS109	-1.0	-1.0	-1.0	
CS101	0.0	0.07426051056745103	-0.9999999926656689	
CS230	-1.0	-1.0	-1.0	
CS300	-1.0	-1.0	-1.0	
CS202	-0.9996017114381104	-0.9768783462650116	-0.6252158520614427	
CS322	-1.0	-1.0	-1.0	
CS109	0.5370495669980353	0.197375320224904	-0.9999999926656689	
CS230	-1.0	-1.0	-1.0	
CS360	-1.0	-1.0	-1.0	
CS330	-0.6640367702678489	0.0	-0.9999999926656689	
CS220	-1.0	-1.0	-1.0	
CS211	-1.0	-1.0	-1.0	
CS311	0.0	0.0	0.0	
CS322	0.5259612776291301	0.15108351434541412	0.8793556667391947	
CS322	0.0	0.0	-0.9999999999876468	
CS101	-1.0	-1.0	-1.0	
CS320	-1.0	-1.0	-1.0	
CS230	0.0	0.0	-0.9598176743921666	
CS211	0.422093427031119	-0.9999999999263838	-1.0	
CS202	0.0	0.0	0.0	
CS101	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS300	0.0	-0.1445329815996905	-0.9999999926656689	
CS320	-1.0	-1.0	-1.0	
CS322	0.0	-0.9999999968283837	-1.0	
CS322	0.0	0.0	0.0	
CS341	0.5259612776291301	-0.9999999993154159	-0.9999999968283837	
CS230	-1.0	-1.0	-1.0	
CS322	-1.0	-1.0	-1.0	
CS322	-0.9013474433988833	-0.9999999999968

CS300	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS380	-1.0	-1.0	-1.0	
CS341	-1.0	-1.0	-1.0	
CS230	-1.0	-1.0	-1.0	
CS300	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS380	-1.0	-1.0	-1.0	
CS442	-1.0	-1.0	-1.0	
CS311	-1.0	-1.0	-1.0	
CS341	-1.0	-1.0	-1.0	
CS492	-1.0	-1.0	-1.0	
CS510	-1.0	-1.0	-1.0	
CS330	-1.0	-1.0	-1.0	
CS101	-1.0	-1.0	-1.0	
CS211	-1.0	-1.0	-1.0	
CS380	-1.0	-1.0	-1.0	
CS330	0.0	0.0	0.0	
CS330	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS206	-1.0	-1.0	-1.0	
CS206	0.0	0.0	0.1878086321677394	
CS204	-1.0	-1.0	-1.0	
CS230	-1.0	-1.0	-1.0	
CS350	-1.0	-1.0	-1.0	
CS300	-0.768276002200737	-0.9999999968283837	-0.9999999968283837	
CS402	-1.0	-1.0	-1.0	
CS492	-1.0	-1.0	-1.0	
CS402	0.5842192731929995	0.0	0.0	
CS230	-1.0	-1.0	-1.0	
CS230	-1.0	-1.0	-1.0	
CS211	-1.0	-1.0	-1.0	
CS300	-1.0	-1.0	-1.0	
CS592	-1.0	-1.0	-1.0	
CS320	-0.45308114570203656	0.1097976239231812	-0.9999999999876468	
CS204	-0.9810744278032283	-0.9999999989393636	-1.0	
CS206	-1.0	-1.0	-1.0	
CS492	-1.0	-1.0	-1.0	
CS

In [40]:
with open("mingyu.txt","w",encoding="utf-8") as out:
    for k,v in pos1.items():
        print(k)
        print(v)

CS402 2016 봄
[[0, 0], [0, 0], [0, 0]]
CS660 2018 봄
[[9.92183748600803, 1], [0, 0], [0, 0]]
CS311 2017 가을
[[0, 0], [6.41447847790136, 1], [2.2985275596975745, 1]]
CS492 2016 가을
[[0, 0], [0, 0], [0, 0]]
CS380 2018 봄
[[0, 0], [0, 0], [0, 0]]
CS341 2014 봄
[[0, 0], [0, 0], [0, 0]]
CS230 2015 봄
[[0, 0], [0, 0], [7.499840953501835, 1]]
CS320 2017 가을
[[5.507187099965201, 1], [0, 0], [0, 0]]
CS204 2015 가을
[[12.5025008474896, 2], [5.370495669980353, 1], [0, 0]]
CS311 2018 봄
[[0, 0], [0, 0], [0, 0]]
CS408 2016 봄
[[0, 0], [5.80471520360587, 1], [0, 0]]
CS206 2017 가을
[[0, 0], [0, 0], [0, 0]]
CS109 2016 봄
[[0, 0], [0, 0], [0, 0]]
CS204 2011 가을
[[0, 0], [0, 0], [0, 0]]
CS300 2012 가을
[[0, 0], [0, 0], [0, 0]]
CS270 2013 봄
[[5.370495669980353, 1], [0, 0], [0, 0]]
CS204 2018 봄
[[0, 0], [0, 0], [0, 0]]
CS310 2014 가을
[[0, 0], [1.097976239231812, 1], [0, 0]]
CS230 2013 봄
[[0, 0], [0, 0], [0, 0]]
CS350 2017 봄
[[5.370495669980353, 1], [0, 0], [0, 0]]
CS206 2014 봄
[[0, 0], [0, 0], [0, 0]]
CS330 2011 가을
[[6.640

In [41]:
#season : 11:13 course:0:5, Year:6:10
# course_dict ={} #key:name, value: dict->key:year+semester val:PNS
# for k,v in PNS.items():
#     name = k[0:5]
#     if name not in course_dict:
#         course_dict[name][k[6:]]=v
# print(course_dict)
# for k1,v1 in course_dict.items():
#     #with codecs.open("../data/line/"+k[0:5]+" line.csv","wb","utf-8-sig") as file:
#     #writer = csv.writer(file, delimiter=',')
#     L = [["Categories","Lecture","Load","Grade"]]
#     for k2,v2 in v1.items():
#         L.append([k2,v2[0][0],v2[1][0],v2[2][0]])
# print(L)

In [42]:
for course_tmp in course_list:
    L =[["Categories","Lecture","Load","Grade"]]
    for i in range(2000,2019):
        for k1,v1 in PNS1.items():
            if k1[0:5] ==course_tmp and k1[6:10] == str(i):
                if k1[11:] == "봄":
                    L.append([k1[6:10]+" Spring",v1[0][0],v1[1][0],v1[2][0]])
                if k1[11:] == "가을":
                    L.append([k1[6:10]+" Fall",v1[0][0],v1[1][0],v1[2][0]])    
    if(len(L)>2):
        for idx in range(1,len(L)-1):
            if int(L[idx][0][0:4])==int(L[idx+1][0][0:4]) and L[idx][0][5:]=="Fall" and L[idx+1][0][5:]=="Spring":
                tmp = L[idx]
                L[idx] = L[idx+1]
                L[idx+1] = tmp
    with open("../data/line/"+course_tmp+" line.csv","w",newline='') as file:
        writer = csv.writer(file, delimiter=',')
        for i in range(len(L)):
            writer.writerow(L[i])
        
        